<a href="https://colab.research.google.com/github/Alecia113/NLP-Ex/blob/main/E9_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise




## E1. Explain the difference between coreference and anaphora with the example. (Your answer will not be marked if you do not add the example)

 用**例子**解释**核心推理**和**拟人**之间的**区别**。如果你不添加例子，你的答案将不会被评分
 coreference ————anaphora

Your answer: coreference： 代指； Hospital ……it ； anaphora：重复， Hospital…… hospital……

## E2. Try Bi-LSTM with CRF!

Now we will apply the Bi-LSTM CRF model we just learned to CoNLL 2003 NER dataset using the pretrained glove embeddings. Please go through and complete the [Function for accuracy] section. 
现在我们将把刚刚学到的Bi-LSTM CRF模型应用于CoNLL 2003 NER数据集，使用预训练的手套嵌入。请浏览并完成[准确性的功能]部分。

### Download Dataset

In [1]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate
drive = None
def authenticate():
    global drive
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)

#Download files
def downloadFiles(fileIds):
    authenticate()
    for fileId in fileIds:    
        downloaded = drive.CreateFile({"id": fileId[1]})
        downloaded.GetContentFile(fileId[0])

In [2]:
#Download file if not existing
try:
  _ = open("train.txt", "r")
except:
  downloadFiles([["train.txt", "1UmNHdUZxjfcuIzCcAKuBvfBXdSWFv47i"]])

try:
  _ = open("validation.txt", "r")
except:
  downloadFiles([["validation.txt", "11bZIh5V9m2nZJ5s5xQ_gxHEHkAEhV8eQ"]])

try:
  _ = open("test.txt", "r")
except:
  downloadFiles([["test.txt", "1V-LQuJWT62aCytYuhZuaxvICsqiF1rdK"]])

In [3]:
def read_data(file_name, n_sample):
    f = open(file_name)
    documents = f.readlines()

    input_data = []
    target_data = []

    temp1 = []
    temp2 = []
    for i in documents:
        if i == '\n':
            input_data.append(temp1)
            target_data.append(temp2)
            temp1 = []
            temp2 = []
        else:
            temp1.append(i.replace('\n','').split(' ')[0].lower())
            temp2.append(i.replace('\n','').split(' ')[3])
    return input_data[:n_sample], target_data[:n_sample]

train_data, target_y_train = read_data("train.txt",400)   # ['york', ',', 'england', '1996-08-22']]一维一句话，二维一个token 
# ['I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],400 ,['I-ORG', 'O', 'I-MISC', 'O', 'O', 'O', 'I-MISC', 'O', 'O']一句话的标签；看起来像IO没看到B
validation_data, target_y_validation = read_data("validation.txt",50) #验证['london', '1996-08-30'],['I-LOC', 'O']
test_data, target_y_test = read_data("test.txt",50) #['nadim', 'ladki']['I-PER', 'I-PER']

In [4]:
#给的例子
print(len(train_data))
print(train_data[1])
print(target_y_train[1])

400
['eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.']
['I-ORG', 'O', 'I-MISC', 'O', 'O', 'O', 'I-MISC', 'O', 'O']


### Preprocess

#### Generate word_to_ix and tag_to_ix

In [5]:
word_to_ix = {}     #进行了一波预处理
for sentence in train_data+validation_data+test_data:
    for word in sentence:
        word = word.lower()
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
word_list = list(word_to_ix.keys())   #2351 token对应inde; index .keys 把关键的元素取出来。就是不要index;按顺序取的感觉

START_TAG = "<START>"
STOP_TAG = "<STOP>"
tag_to_ix = {START_TAG:0, STOP_TAG:1}     #如下 ；还是BIO
for tags in target_y_train+target_y_validation:     #['O', 'O', 'O', 'I-ORG']
    for tag in tags:      #每一个的词的tag
        if tag not in tag_to_ix:
            tag_to_ix[tag] = len(tag_to_ix) #就是又匹配上重新复制了。

'''
{'<START>': 0,
 '<STOP>': 1,
 'B-LOC': 7,
 'B-MISC': 8,
 'I-LOC': 6,
 'I-MISC': 4,
 'I-ORG': 3,
 'I-PER': 5,
 'O': 2}
 '''

"\n{'<START>': 0,\n '<STOP>': 1,\n 'B-LOC': 7,\n 'B-MISC': 8,\n 'I-LOC': 6,\n 'I-MISC': 4,\n 'I-ORG': 3,\n 'I-PER': 5,\n 'O': 2}\n "

#### Generate Embedding Matrix 把word变成word-embedding

In [6]:
import gensim.downloader as api
import numpy as np   #要加的
word_emb_model = api.load("glove-twitter-25") 

EMBEDDING_DIM = 25      #补充到25了

embedding_matrix = []
for word in word_list:
    try:
        embedding_matrix.append(word_emb_model.wv[word])
    except:
        embedding_matrix.append([0]*EMBEDDING_DIM)
embedding_matrix = np.array(embedding_matrix) #熟悉的节奏lookup tablel; 2维度
embedding_matrix.shape            #(2351, 25)

[==================================================] 100.0% 104.8/104.8MB downloaded


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # Remove the CWD from sys.path while we load stuff.


(2351, 25)

#### convert dataset into idxs   将数据集转换为idxs

In [7]:
def to_index(data, to_ix):
    input_index_list = []
    for sent in data:         #sent 每句话
        input_index_list.append([to_ix[w] for w in sent])       #w是每个词    查询：【】词加'' word_to_ix['near'] 返回的是每个词的ind
    return input_index_list         # 把每个词的inde，又丢进list，一句话都是由inde组成，把这个数据集全都改成了index的模式

train_input_index =  to_index(train_data,word_to_ix)    # 句子toke，词字典
train_output_index = to_index(target_y_train,tag_to_ix)   #每句没词类别，BIO类别典
val_input_index = to_index(validation_data,word_to_ix)    # 句子toke，词字典
val_output_index = to_index(target_y_validation,tag_to_ix)    #每句没词类别，BIO类别典
test_input_index = to_index(test_data,word_to_ix)   # 句子toke，词字典
test_output_index = to_index(target_y_test,tag_to_ix)   #每句没词类别，BIO类别典

### Model

In [76]:
'''
随机数据

为CPU中设置种子，生成随机数
torch.manual_seed(number)   用的这
为特定GPU设置种子，生成随机数
torch.cuda.manual_seed(number)
为所有GPU设置种子，生成随机数
torch.cuda.manual_seed_all()

在需要生成随机数据的实验中，每次实验都需要生成数据。设置随机种子是为了确保每次生成固定的随机数，这就使得每次实验结果显示一致了，有利于实验的比较和改进。
'''
'''tag_to_ix
{'<START>': 0,
 '<STOP>': 1,
 'B-LOC': 7,
 'B-MISC': 8,
 'I-LOC': 6,
 'I-MISC': 4,
 'I-ORG': 3,
 'I-PER': 5,
 'O': 2}
 '''

In [9]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)      #手册

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 50

model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM).to(device)    #2351, 类别的词典， 50 隐藏原。
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

In [77]:
class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):    #vocab_size = 2351；字典里有多少词。类别词典 25最长embedding-长度（每个词）；hidden隐藏元50
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)     #传参了一波

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        """Here we use the embedding matrix as the initial weights of nn.Embedding"""""#这里我们使用嵌入矩阵作为nn.Embedding的初始权重""
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2, num_layers=1, bidirectional=True)#" // " 表示整数除法

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()



In [82]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)      #手册
#最大值，最大分数
def argmax(vec):      
    # return the argmax as a python int# 将argmax作为一个Python int返回
    _, idx = torch.max(vec, 1)
    return idx.item()     #item() 方法是用来将只有一个元素的numpy数组或tensor张量转化为标量的方法
                          #Python 字典 items() 函数以列表返回可遍历的(键, 值) 元组数组。


# Compute log sum exp in a numerically stable way for the forward algorithm 以数字上稳定的方式计算正向算法的对数和exp 
#log_sum_exp(next_tag_var).view(1))
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2).to(device),
                torch.randn(2, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

#### Function for accuracy [Please Complete this part]

Please complete the *cal_acc* function that generates the model predictions using the input data and calculates the accuracy by comparing the model predictions with the ground truth labels. You can refer to the [Train the model] section regarding what the inputs and outputs are and how they will be used.

**Hint**: You are going to test the "model" using "input_index"(whose shape is (num_samples, seq_length)), and you are going to return three variables:
- **predicted**: This should be a list, and each item in the list should be the predicted NER label for a word. Suppose you have two sentences, sentence 1 has 8 words and sentence 2 has 6 words. This list should have 14(=8+6) items.
- **ground_truth**: This should also be a list, and each item in the list should be the actual NER label for a word, which can be easily gotten from "output_index". Suppose you have two sentences, sentence 1 has 8 words and sentence 2 has 6 words. This list should have 14(=8+6) items.
- **accuracy**: You are going to use "predicted" and "ground_truth" to calculate the accuracy.


请完成cal_acc函数，该函数使用输入数据生成模型预测，并通过比较模型预测和地面真实标签来计算准确性。关于输入和输出是什么以及如何使用它们，你可以参考[训练模型]部分。

提示：你将使用 "input_index"（其形状为（num_samples，seq_length））来测试 "模型"，你将返回三个变量。

预测的。这应该是一个列表，列表中的每一项都应该是一个词的预测的NER标签。假设你有两个句子，句子1有8个词，句子2有6个词。这个列表应该有14（=8+6）项。
ground_truth：这也应该是一个列表，列表中的每一项都应该是一个词的实际NER标签，这可以很容易从 "output_index "中得到。假设你有两个句子，句子1有8个词，句子2有6个词。这个列表应该有14（=8+6）项。
准确性：你要用 "预测 "和 "ground_truth "来计算准确性。


In [ ]:
# Check predictions before training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    precheck_tags = torch.tensor([tag_to_ix[t] for t in training_data[0][1]], dtype=torch.long)
    print(model(precheck_sent))

 
# Check predictions after training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    print(model(precheck_sent))
# We got it
#https://blog.csdn.net/kyle1314608/article/details/100589398?ops_request_misc=&request_id=&biz_id=102&utm_term=python%20Bi-LSTM%20with%20CRF&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduweb~default-8-.first_rank_v2_pc_rank_v29&spm=1018.2226.3001.4187

In [ ]:
import numpy as np
def cal_acc(model, input_index, output_index):

    return predicted, ground_truth, accuracy

Initialising the model 初始化模式

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 50

model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

#### Train the model

In [ ]:
"""Each epoch will take about 1-2 minutes"""

import datetime

for epoch in range(20):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(val_input_index):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

# The log below is the sample output for this section

Epoch:1, Training loss: 3404.31, train acc: 0.8501, val loss: 437.62, val acc: 0.8032, time: 47.32s
Epoch:2, Training loss: 2076.04, train acc: 0.8808, val loss: 425.70, val acc: 0.8157, time: 47.92s
Epoch:3, Training loss: 1580.79, train acc: 0.9072, val loss: 442.27, val acc: 0.8244, time: 47.64s
Epoch:4, Training loss: 1328.45, train acc: 0.9104, val loss: 469.88, val acc: 0.8232, time: 47.37s
Epoch:5, Training loss: 1150.15, train acc: 0.9247, val loss: 480.14, val acc: 0.8281, time: 48.19s
Epoch:6, Training loss: 1018.71, train acc: 0.9315, val loss: 482.81, val acc: 0.8281, time: 47.43s
Epoch:7, Training loss: 892.29, train acc: 0.9363, val loss: 506.21, val acc: 0.8281, time: 47.94s
Epoch:8, Training loss: 791.39, train acc: 0.9434, val loss: 520.18, val acc: 0.8219, time: 47.44s
Epoch:9, Training loss: 679.52, train acc: 0.9499, val loss: 517.10, val acc: 0.8381, time: 47.88s
Epoch:10, Training loss: 599.38, train acc: 0.9578, val loss: 540.84, val acc: 0.8356, time: 47.33s
Epo

### Testing

In [ ]:
# Call the cal_acc functions you implemented as required
y_pred, y_true, _ = cal_acc(model,test_input_index,test_output_index)

def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]

y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true_decode,y_pred_decode,digits=4))
# The log below is the sample output for this section

              precision    recall  f1-score   support

       I-LOC     0.8750    0.9333    0.9032        45
      I-MISC     0.8800    0.8800    0.8800        25
       I-ORG     0.0000    0.0000    0.0000         3
       I-PER     0.9135    0.7540    0.8261       126
           O     0.9635    0.9839    0.9736       806

    accuracy                         0.9473      1005
   macro avg     0.7264    0.7102    0.7166      1005
weighted avg     0.9483    0.9473    0.9467      1005

